In [ ]:
import socket,cv2,threading
import numpy as np

sock = socket.socket()
print(socket.gethostbyname(socket.gethostname()))
sock.bind((socket.gethostbyname(socket.gethostname()), 1234))
sock.listen()
session, address = sock.accept()
print(session.recv(4*1024)) 
cameraIndex = 'http://192.168.0.121:8080/video'
camera = cv2.VideoCapture(cameraIndex) 

def sender():
    while True:
        status, photo = camera.read()
        photo = cv2.resize(photo, (640, 480))
        if status:
            session.send(np.ndarray.tobytes(photo))
        else: print("Not in frame")

def receiver():
    print("Entered")
    while True:
        data = session.recv(10000000000)
        if(data == b'finished'):
            print("Finished")
            camera.release()
            session.close()
        else:
            photo =  np.frombuffer(data, dtype=np.uint8)
            if len(photo) == 640*480*3:
                cv2.imshow('From Client side', photo.reshape(480, 640, 3))
                if cv2.waitKey(80) == 13:
                    session.send(b'finished')
                    camera.release()
                    cv2.destroyAllWindows()
                    exit()
                
threading.Thread(target=sender).start()
threading.Thread(target=receiver).start()
